In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(130298) # to have the same random numbers

In [2]:
%pwd

'/Users/annachiararossi/Documents/Healthcare data/heart-failure-project'

In [3]:
drugs = pd.read_csv('dat_md.csv')
drugs.drop(columns = ['Unnamed: 0'], inplace = True)
drugs.shape

(12654, 2)

We notice that this dataset has much more rows than the other: so we want to check if there are patients for which there is more than one row in the table (i.e. patients that take more than one drug). There are indeed a lot of them (10650).

In [4]:
drugs['inpatient.number'].duplicated().sum()

10650

In [5]:
drugs.head()

,inpatient.number,Drug_name
0,857781,sulfotanshinone sodium injection
1,857781,Furosemide tablet
2,857781,Meglumine Adenosine Cyclophosphate for injection
3,857781,Furosemide injection
4,857781,Milrinone injection


We observe the different drugs administrated to the patients.

In [6]:
drugs['Drug_name'].value_counts()

Spironolactone tablet                               1833
Furosemide injection                                1718
Furosemide tablet                                   1641
Meglumine Adenosine Cyclophosphate for injection    1114
Deslanoside injection                               1015
Digoxin tablet                                       998
Atorvastatin calcium tablet                          822
Milrinone injection                                  707
sulfotanshinone sodium injection                     570
Benazepril hydrochloride tablet                      434
Valsartan Dispersible tablet                         348
Shenfu injection                                     338
Isosorbide Mononitrate Sustained Release tablet      326
Hydrochlorothiazide tablet                           283
Torasemide tablet                                    252
Nitroglycerin injection                              203
Isoprenaline Hydrochloride injection                  30
Dobutamine hydrochloride inject

First, we consider only one category for the Furosemide given by injection or by tablet, since it is indeed the same drug, only subministrated in different ways.

In [7]:
new_col = drugs['Drug_name'].copy()
new_col[new_col == 'Furosemide injection'] = 'Furosemide'
new_col[new_col == 'Furosemide tablet'] = 'Furosemide'
drugs['Drug_name'] = new_col

Now we remove the rows corresponding to the two drugs taken only by a very small number of persons, since we will not be able to infer something with so few patients taking them.

In [8]:
too_few1 = list(drugs.index[drugs['Drug_name'] == 'Isoprenaline Hydrochloride injection'])
too_few2 = list(drugs.index[drugs['Drug_name'] == 'Dobutamine hydrochloride injection'])

def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

too_few = union(too_few1, too_few2)
drugs.drop(too_few, inplace = True)
drugs.reset_index(inplace=True, drop=True)

In [9]:
sum(drugs.duplicated())

1446

Checking for duplicated rows, we notice that now there are 1446 repeated rows: these will correspond to the patients taking Furosemide both by injection and by tablet. So we remove these duplicates.

In [10]:
drugs.drop_duplicates(inplace=True)
drugs.reset_index(inplace=True, drop=True)
drugs

,inpatient.number,Drug_name
0,857781,sulfotanshinone sodium injection
1,857781,Furosemide
2,857781,Meglumine Adenosine Cyclophosphate for injection
3,857781,Milrinone injection
4,857781,Deslanoside injection
...,...,...
11151,791864,Spironolactone tablet
11152,791864,Valsartan Dispersible tablet
11153,791864,Digoxin tablet
11154,791864,Deslanoside injection


# Grouping drugs according to their aim

We try to group the different drugs according to their aim and/or their acting principle.
From literature, we found that these drugs can be subdivided into 5 main groups:
- diuretics: 'Furosemide', 'Spironolactone tablet', 'Hydrochlorothiazide tablet', 'Torasemide tablet'
- anti-hypertension: 'Spironolactone tablet', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet'
- heart failure: 'Meglumine Adenosine Cyclophosphate for injection', 'Deslanoside injection', 'Shenfu injection', 'Nitroglycerin injection'
- angina and other cardiac problems: 'Meglumine Adenosine Cyclophosphate for injection', 'Digoxin tablet', 'Milrinone injection', 'sulfotanshinone sodium injection', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet', 'Isosorbide Mononitrate Sustained Release tablet', 'Nitroglycerin injection'
- lowering lipids: 'Atorvastatin calcium tablet'

In [11]:
drugs_by_aim=drugs.copy()

In [12]:
DIURETICS= ['Furosemide', 'Spironolactone tablet', 'Hydrochlorothiazide tablet', 'Torasemide tablet']
HYPERTENSION = ['Spironolactone tablet', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet']
HEART_FAILURE = ['Meglumine Adenosine Cyclophosphate for injection', 'Deslanoside injection', 'Shenfu injection',
                 'Nitroglycerin injection']
ANGINA_ETAL = ['Meglumine Adenosine Cyclophosphate for injection', 'Digoxin tablet', 'Milrinone injection',
               'sulfotanshinone sodium injection', 'Benazepril hydrochloride tablet', 
               'Valsartan Dispersible tablet', 'Isosorbide Mononitrate Sustained Release tablet', 
               'Nitroglycerin injection']
CHOLESTEROL = ['Atorvastatin calcium tablet']

# create a list containing these groups of drugs

list_of_groups = [DIURETICS, HYPERTENSION, HEART_FAILURE, ANGINA_ETAL, CHOLESTEROL]

In [13]:
drug_group = ['diuretics','hypertension','heart_failure','angina_etal','cholesterol']

for group_idx,drug in enumerate(drug_group):
    
    new_col = drugs_by_aim['Drug_name'].copy()
    
    for i in range(drugs.shape[0]):
        if new_col[i] in list_of_groups[group_idx]:
            new_col[i]=1
        else:
            new_col[i]=0
            
    drugs_by_aim[drug]= new_col

drugs_by_aim.drop('Drug_name', axis=1, inplace=True)


We also drop the column where we specify the particular name of the drugs and we only keep into account their aim.

In [14]:
drugs_by_aim[drugs_by_aim['inpatient.number'] == 857781]

,inpatient.number,diuretics,hypertension,heart_failure,angina_etal,cholesterol
0,857781,0,0,0,1,0
1,857781,1,0,0,0,0
2,857781,0,0,1,1,0
3,857781,0,0,0,1,0
4,857781,0,0,1,0,0
5,857781,1,0,0,0,0
6,857781,0,1,0,1,0
7,857781,0,0,0,0,1
8,857781,0,0,0,1,0


We don't need to drop duplicates, but we need to get from each patient ID a row having 1 if the patient has at least a 1 in that column, 0 otherwise. This can simply be achieved by taking the max value for each patient on each column.

In [15]:
df_grouped = drugs_by_aim.groupby(['inpatient.number'])
df_grouped.ngroups

2004

In [16]:
drugs_by_patient = df_grouped.max()
drugs_by_patient.head()

,diuretics,hypertension,heart_failure,angina_etal,cholesterol
inpatient.number,,,,,
722128,1,1,1,1,0
723327,1,1,1,1,1
723617,1,1,0,0,1
724385,1,1,1,1,0
725509,1,1,1,1,0


Now we want to merge these two datasets over the patient ID (based on the patient ID in the main dataframe, because some patients have already been removed for other reasons). Notice that the number of patients in this dataset is 2004, while in the original dataset we have 2008. Thus, there should be 4 patients which are not registered because they're not taking any drug.

In [17]:
X_train = pd.read_csv('train_data.csv')
X_train.set_index('inpatient.number', inplace = True)
X_train.head()

,DestinationDischarge,admission.ward,admission.way,discharge.department,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,...,direct.bilirubin,total.bile.acid,total.protein,cholesterol,low.density.lipoprotein.cholesterol,triglyceride,high.density.lipoprotein.cholesterol,GCS,dischargeDay,ageCat
inpatient.number,,,,,,,,,,,,,,,,,,,,,
734179,Home,Cardiology,NonEmergency,Cardiology,0,36.3,80.0,19.0,130.0,90.0,...,14.3,14.0,75.1,3.30,1.87,0.64,0.92,15,13,"(59,69]"
772351,Home,Cardiology,Emergency,Cardiology,1,36.6,83.0,19.0,110.0,70.0,...,2.9,3.9,66.2,4.24,2.58,1.03,1.12,15,7,"(49,59]"
836826,Home,Cardiology,NonEmergency,Cardiology,1,36.0,96.0,18.0,100.0,60.0,...,16.9,1.7,65.8,2.44,1.00,0.62,0.95,15,7,"(69,79]"
764082,Home,Cardiology,Emergency,Cardiology,0,37.0,90.0,19.0,150.0,58.0,...,2.8,NaN,79.4,2.85,1.58,1.31,0.85,15,5,"(69,79]"
789038,HealthcareFacility,Cardiology,NonEmergency,Cardiology,0,36.0,82.0,21.0,132.0,90.0,...,10.6,5.1,74.0,3.30,1.52,2.13,0.98,15,8,"(59,69]"


In [18]:
def merge_drugs(data,drugs_by_patient):
    # suffix is needed because we already have a variable called cholesterol
    data_final = data.join(drugs_by_patient, rsuffix = '_drug')
    # convert the last five columns to int again
    data_final.iloc[:,-5:] = data_final.iloc[:,-5:].astype('Int64')
    return data_final

In [19]:
X_train = merge_drugs(X_train,drugs_by_patient)
X_train.head()

,DestinationDischarge,admission.ward,admission.way,discharge.department,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,...,triglyceride,high.density.lipoprotein.cholesterol,GCS,dischargeDay,ageCat,diuretics,hypertension,heart_failure,angina_etal,cholesterol_drug
inpatient.number,,,,,,,,,,,,,,,,,,,,,
734179,Home,Cardiology,NonEmergency,Cardiology,0,36.3,80.0,19.0,130.0,90.0,...,0.64,0.92,15,13,"(59,69]",1,1,1,1,0
772351,Home,Cardiology,Emergency,Cardiology,1,36.6,83.0,19.0,110.0,70.0,...,1.03,1.12,15,7,"(49,59]",1,1,1,1,0
836826,Home,Cardiology,NonEmergency,Cardiology,1,36.0,96.0,18.0,100.0,60.0,...,0.62,0.95,15,7,"(69,79]",1,1,0,0,0
764082,Home,Cardiology,Emergency,Cardiology,0,37.0,90.0,19.0,150.0,58.0,...,1.31,0.85,15,5,"(69,79]",1,1,0,1,0
789038,HealthcareFacility,Cardiology,NonEmergency,Cardiology,0,36.0,82.0,21.0,132.0,90.0,...,2.13,0.98,15,8,"(59,69]",1,1,1,1,0


In [20]:
print(f'Patients not taking drugs: {np.where(X_train["diuretics"].isnull())[0]}')

Patients not taking drugs: [ 472 1144 1347 1535]


In [21]:
X_train.iloc[:,-5:] = X_train.iloc[:,-5:].fillna(0)

In [22]:
X_train.iloc[:,-5:].sum()

diuretics           1550
hypertension        1453
heart_failure       1286
angina_etal         1467
cholesterol_drug     631
dtype: int64

In [23]:
for i in range(6):
    print(f'Number of patients taking {i} drugs: {len(np.where(X_train.iloc[:,-5:].sum(axis=1)==i)[0])}')

Number of patients taking 0 drugs: 4
Number of patients taking 1 drugs: 6
Number of patients taking 2 drugs: 66
Number of patients taking 3 drugs: 192
Number of patients taking 4 drugs: 847
Number of patients taking 5 drugs: 457


In [24]:
X_test = pd.read_csv('test_data.csv')
X_test.set_index('inpatient.number', inplace = True)
X_test.head()

,DestinationDischarge,admission.ward,admission.way,discharge.department,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,...,direct.bilirubin,total.bile.acid,total.protein,cholesterol,low.density.lipoprotein.cholesterol,triglyceride,high.density.lipoprotein.cholesterol,GCS,dischargeDay,ageCat
inpatient.number,,,,,,,,,,,,,,,,,,,,,
806277,HealthcareFacility,GeneralWard,Emergency,Cardiology,0,37.1,108.0,21.0,130.0,75.0,...,11.9,8.4,70.6,4.00,1.48,1.13,1.53,11,19,"(79,89]"
801945,Home,Cardiology,Emergency,Cardiology,1,36.8,98.0,19.0,130.0,64.0,...,8.3,9.2,69.2,2.19,0.82,0.87,0.93,15,10,"(79,89]"
807741,NaN,Cardiology,NonEmergency,Cardiology,0,36.3,60.0,18.0,126.0,64.0,...,16.6,NaN,79.1,3.40,1.46,0.85,1.24,15,9,"(49,59]"
742279,Home,Others,Emergency,Cardiology,0,36.4,56.0,18.0,105.0,65.0,...,16.4,NaN,76.7,3.01,1.14,0.48,1.27,15,11,"(59,69]"
801155,Home,Cardiology,Emergency,Cardiology,1,36.9,114.0,22.0,136.0,88.0,...,6.3,4.4,77.9,5.75,3.18,1.23,1.19,15,12,"(69,79]"


In [25]:
X_test = merge_drugs(X_test,drugs_by_patient)
X_test.head()

,DestinationDischarge,admission.ward,admission.way,discharge.department,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,...,triglyceride,high.density.lipoprotein.cholesterol,GCS,dischargeDay,ageCat,diuretics,hypertension,heart_failure,angina_etal,cholesterol_drug
inpatient.number,,,,,,,,,,,,,,,,,,,,,
806277,HealthcareFacility,GeneralWard,Emergency,Cardiology,0,37.1,108.0,21.0,130.0,75.0,...,1.13,1.53,11,19,"(79,89]",1,1,1,1,1
801945,Home,Cardiology,Emergency,Cardiology,1,36.8,98.0,19.0,130.0,64.0,...,0.87,0.93,15,10,"(79,89]",1,1,1,1,0
807741,NaN,Cardiology,NonEmergency,Cardiology,0,36.3,60.0,18.0,126.0,64.0,...,0.85,1.24,15,9,"(49,59]",1,1,0,1,0
742279,Home,Others,Emergency,Cardiology,0,36.4,56.0,18.0,105.0,65.0,...,0.48,1.27,15,11,"(59,69]",1,1,0,1,1
801155,Home,Cardiology,Emergency,Cardiology,1,36.9,114.0,22.0,136.0,88.0,...,1.23,1.19,15,12,"(69,79]",1,1,1,1,0


In [26]:
X_test.iloc[:,-5:].sum()

diuretics           389
hypertension        374
heart_failure       326
angina_etal         378
cholesterol_drug    176
dtype: int64

In [27]:
for i in range(6):
    print(f'Number of patients taking {i} drugs in test set: {len(np.where(X_test.iloc[:,-5:].sum(axis=1)==i)[0])}')

Number of patients taking 0 drugs in test set: 0
Number of patients taking 1 drugs in test set: 6
Number of patients taking 2 drugs in test set: 7
Number of patients taking 3 drugs in test set: 44
Number of patients taking 4 drugs in test set: 214
Number of patients taking 5 drugs in test set: 127


In [28]:
X_train.to_csv('train_data_drugs.csv')

In [29]:
X_test.to_csv('test_data_drugs.csv')